<a href="https://colab.research.google.com/github/yutawatabe/quantiative_trade_lecture/blob/main/Computing_Eaton_Kortum_exact_hat_algebra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exact hat algebraを用いた反実仮想の計算

In [ ]:
# numpy, mathをインポート
import numpy as np # 行列計算のライブラリ
from math import gamma # 価格指数のため

パラメーターをシミュレートして、Eaton Kortumモデルの均衡を計算する。均衡を計算するために賃金を更新する関数を再び導入し、最終的に均衡の賃金、貿易額を求めるところまで関数化する。

In [ ]:
def updatewage(w,theta,sigma,N,L,T,ｄ,psi):
  Xn = w * L

  pi = np.zeros((N,N)) # 輸入シェア（n国の総消費のうち、i国の財が何割を占めるか）
  pi_num = np.zeros((N,N)) # 輸入シェアの分子部分
  Phi = np.zeros((N)) # 輸入シェアの分母部分

  for OR,DE in np.ndindex((N,N)):
    pi_num[OR,DE] = T[OR] * (w[OR] * ｄ[OR,DE]) ** (-theta)
    Phi[DE] += pi_num[OR,DE] # Phi[DE] = pi_den[DE] + pi_num[OR,DE]

  for OR,DE in np.ndindex((N,N)):
    pi[OR,DE] = pi_num[OR,DE] / Phi[DE]

  P = gamma((theta + sigma - 1) / theta) **(1/(1-sigma)) * Phi ** (-1/theta)

  X = np.zeros((N,N))
  for OR,DE in np.ndindex((N,N)):
    X[OR,DE] = pi[OR,DE] * Xn[DE]

  L_S = L
  L_D = np.zeros((N))
  for OR,DE in np.ndindex((N,N)):
    L_D[OR] += X[OR,DE] / w[OR] # L_D[OR] = L_D[OR] + X[OR,DE] / w[OR]

  Z = L_D - L_S
  w_new = w * (1 + psi * (Z / L) )

  wgdp = np.sum(w_new * L)
  w_new = w_new / wgdp

  return w_new,Z,X,P

def solve_eqm(theta,sigma,N,L,T,d,psi,tol,maxiter):
  Z = np.ones((N))
  w = np.ones((N))

  iter = 1
  while max(np.abs(Z)) > tol and iter < maxiter:
    iter += 1
    w_old = np.copy(w)
    w,Z,_,_ = updatewage(w,theta=4,sigma=3,N=3,L=L,T=T,d=d,psi=0.1)
    if iter % 10 == 0:
      print(iter)
      #print(Z)
      #print(w)

  if iter == maxiter:
    print("Not done")
    return w,Z,P,X
  else:
    w,Z,X,P = updatewage(w,theta=4,sigma=3,N=3,L=L,T=T,d=d,psi=0.1)
    return w,Z,P,X


この関数を用いて、均衡をパラメーターから計算しておく。

In [ ]:
N = 3 # 国の数
theta = 4 # 貿易弾力性のパラメーター
sigma = 3 # 代替の弾力性のパラメーター
T = np.array([1., 1., 1.]) # 技術のパラメーター
L = np.array([1,1.5,1.5]) # 人口
d = np.ones((N,N)) * 1.5 # 貿易費用
for OR in np.ndindex((N)):
  d[OR,OR] = 1 #国内貿易費用は

w_sq,_,P_sq,X_sq = solve_eqm(theta,sigma,N,L,T,d,psi=0.1,tol=0.001,maxiter=1000)
print(w_sq)

10
[0.26058948 0.24647017 0.24647017]


この数値を初期均衡として、反実仮想を計算する。まずパラメーターが既知として、モデルを解きなおしてみる。ここでは貿易自由化を考える。初期均衡では
$$ \tau_{in} = 1.5 $$
だったところを
$$ \tau_{in}' = 1.2 $$
としておく（国内貿易費用は1のまま固定）。

In [ ]:
d_new = np.ones((N,N)) * 1.2 # 貿易費用
for OR in np.ndindex((N)):
  d_new[OR,OR] = 1 #国内貿易費用は 1
print(d_new)

dhat = d_new / d
print(dhat)

[[1.  1.2 1.2]
 [1.2 1.  1.2]
 [1.2 1.2 1. ]]
[[1.  0.8 0.8]
 [0.8 1.  0.8]
 [0.8 0.8 1. ]]


モデルを解きなおしてみて、実質賃金（厚生）を比較すると…。

In [ ]:
w_cf,_,P_cf,X_cf = solve_eqm(theta,sigma,N,L,T,d_new,psi=0.1,tol=0.001,maxiter=1000)

U_sq = w_sq / P_sq
U_cf = w_cf / P_cf
print(U_cf/U_sq)

10
[1.10938209 1.08093633 1.08093633]


Exact hat algebraを使って、このTrade liberizationの反実仮想が整合的に求まるかを確認する。具体的には $(w_{sq} / P_{sq}) / (w_{cf} / P_{cf})$という実質賃金がモデルを解きなおした場合と、Exact hat algebraを使った場合で整合的かを確認していく。

Exact hat algebraのために、総消費と輸入シェアを計算する。

In [ ]:
Xabs_sq = np.sum(X_sq,axis=0)

pi_sq = np.zeros((N,N))
for OR,DE in np.ndindex((N,N)):
  pi_sq[OR,DE] = X_sq[OR,DE] / Xabs_sq[DE]
print(X_sq)
print(Xabs_sq)
print(pi_sq)

[[0.17447018 0.04312279 0.04312279]
 [0.04305264 0.27271905 0.05387043]
 [0.04305264 0.05387043 0.27271905]]
[0.26057545 0.36971227 0.36971227]
[[0.66955724 0.11663879 0.11663879]
 [0.16522138 0.73765214 0.14570907]
 [0.16522138 0.14570907 0.73765214]]


とりあえず$\boldsymbol{\hat{w}}$をすべて1と仮置きして、Exact hat algebraの均衡が成立しているかを確認する。

In [ ]:
what = np.ones((N))
print(what)

[1. 1. 1.]


総消費もこれに伴って変化するため、それを計算しておく
$$ X_{abs,i}' = \hat{w}_{i} X_{abs,i} $$

In [ ]:
Xabs1 = what * Xabs_sq
print(Xabs1)

[0.26057545 0.36971227 0.36971227]


この$\boldsymbol{\hat{w}}$から$\boldsymbol{\hat{\pi}}$を計算する
$$ \hat{\pi}_{ni} = \frac{(\hat{w}_i \hat{d}_{ni})^{-\theta}}{ \sum_{k=1}^N \pi_{nk} (\hat{w}_k \hat{d}_{nk})^{-\theta} } $$
そして価格指数も計算する
$$ \hat{P}_n = \left( \sum_{k=1}^N \pi_{nk} (\hat{w}_k \hat{d}_{nk})^{-\theta} \right)^{-1/\theta} $$

In [ ]:
pihat = np.zeros((N,N))
pihat_num = np.zeros((N,N))
pihat_den = np.zeros((N))

for OR,DE in np.ndindex((N,N)):
  pihat_num[OR,DE] = (what[OR] * dhat[OR,DE] )**(-theta)
  pihat_den[DE] += pi_sq[OR,DE] * pihat_num[OR,DE]

for OR,DE in np.ndindex((N,N)):
  pihat[OR,DE] = pihat_num[OR,DE] / pihat_den[DE]

Phat = pihat_den ** (-1/theta)

print(pihat)
print(pihat * pi_sq)
print(np.sum(pihat*pi_sq,axis=0))

[[0.66003744 1.70645845 1.70645845]
 [1.68884667 0.73254998 1.78845211]
 [1.68884667 1.78845211 0.73254998]]
[[0.44193285 0.19903925 0.19903925]
 [0.27903357 0.54036707 0.26059369]
 [0.27903357 0.26059369 0.54036707]]
[1. 1. 1.]


ここで新しい貿易額を計算しておく
$$ X'_{ni} = \pi_{ni} \hat{\pi}_{ni} X'_{abs,i} $$

In [ ]:
X1 = np.zeros((N,N))
for OR,DE in np.ndindex((N,N)):
  X1[OR,DE] = pi_sq[OR,DE] * pihat[OR,DE] * Xabs1[DE]


仮置きされた$ \boldsymbol{\hat{w}}$の下で、労働市場均衡が成立しているかを確認する。労働市場が均衡しているということは

1.   労働供給は変化していない。
2.   労働市場が均衡するためには、労働需要も変化してはいけない。

つまり
$$ L'_{D,i} = \frac{\sum_{n=1}^N X_{ni}'}{w_i'} $$
について
$$ L'_{D,i} = L_{D,i} = L_{S,i} = L_i$$
となっていなくてはならない。これをhatで表現すると
$$ \hat{w}_i w_i L_{i} = \sum_{n=1}^N X_{ni}' $$
左を労働供給、右を労働需要ととらえ、この差を超過需要として計算してみる

In [ ]:
wL_D = np.zeros((N))
wL_S = w_sq * what * L

for OR,DE in np.ndindex((N,N)):
  wL_D[OR] += X1[OR,DE]

Z = (wL_D - wL_S) / what
print(Z)

[ 0.01037596 -0.00518798 -0.00518798]


モデルを解くのと同様に超過需要から$\boldsymbol{\hat{w}}$をアップデートする。同様に$\psi$を設定しておく。
$$ \hat{w}_{new,i} = \hat{w}_i * \left(1 + \psi * \frac{Z_i}{X'_{abs,i}}\right) $$

In [ ]:
psi = 0.1 # 収束のスピードをコントロールするパラメーター

what_new = what * (1 + psi * (Z / Xabs1) )

ここで新しい$\boldsymbol{\hat{w}}$と古い$\boldsymbol{\hat{w}}$を比較する。ちゃんと超過需要が発生していた国において賃金が高くなっているか確認できるはず。

In [ ]:
print(Z)
print(what_new - what)

[ 0.01037596 -0.00518798 -0.00518798]
[ 0.00398194 -0.00140325 -0.00140325]


均衡賃金の変化は基準化なしに一意にはきまらない。最後に反実仮想の世界のGDPが1になる（世界全体のGDPが変化しない）ように基準化しておく。

In [ ]:
wgdp = np.sum(what_new * Xabs_sq)
print(wgdp)
what_new = what_new / wgdp
print(what_new)

1.0
[1.00398194 0.99859675 0.99859675]


いままでの流れをまとめると


1.   パラメーターを設定する
2.   仮定した賃金変化をもとに、輸入シェアの変化、貿易額の変化を計算する
3.   新しい貿易額を用いて、労働市場の均衡をチェックする
4.   労働の超過需要を用いて、仮置きした賃金変化を更新していく。

この2から4を**関数**にまとめる。

In [ ]:
def updatewagehat(what,theta,N,X,psi):

  #総需要を計算する
  Xabs_sq = np.sum(X_sq,axis=0)
  Xabs1 = what * Xabs_sq

  # 輸入シェアを計算する
  pi_sq = np.zeros((N,N))
  for OR,DE in np.ndindex((N,N)):
    pi_sq[OR,DE] = X_sq[OR,DE] / Xabs_sq[DE]

  # 輸入シェアの変化を計算する
  pihat = np.zeros((N,N))
  pihat_num = np.zeros((N,N))
  pihat_den = np.zeros((N))
  for OR,DE in np.ndindex((N,N)):
    pihat_num[OR,DE] = (what[OR] * dhat[OR,DE] )**(-theta)
    pihat_den[DE] += pi_sq[OR,DE] * pihat_num[OR,DE]
  for OR,DE in np.ndindex((N,N)):
    pihat[OR,DE] = pihat_num[OR,DE] / pihat_den[DE]

  # 価格指数の変化を計算する
  Phat = pihat_den ** (-1/theta)

  # 新しい貿易額を計算する
  X1 = np.zeros((N,N))
  for OR,DE in np.ndindex((N,N)):
    X1[OR,DE] = pi_sq[OR,DE] * pihat[OR,DE] * Xabs1[DE]

  # 新しい均衡での労働市場均衡を計算する
  wL_D = np.zeros((N))
  wL_S = w_sq * what * L

  for OR,DE in np.ndindex((N,N)):
    wL_D[OR] += X1[OR,DE]

  Z = (wL_D - wL_S) / what
  what_new = what * (1 + psi * (Z / Xabs1) )
  wgdp = np.sum(Xabs1)
  what_new = what_new / wgdp

  return what_new,Z,Phat,X1

ここでちゃんと結果が前の関数なしで書いたiterationと一致するかを確認する。

In [ ]:
what_newfunc,_,_,_ = updatewagehat(what,theta=theta,N=N,X=X_sq,psi=psi)
print(what_newfunc)
print(what_new)

[1.00870938 0.99693072 0.99693072]
[1.00398194 0.99859675 0.99859675]


最後にwhile-loopを用いて$\boldsymbol{\hat{w}}$が均衡条件を満たすまで更新していく

In [ ]:
tol = 0.0001
iter = 1

Z = np.ones((N))
what = np.ones((N))
while max(np.abs(Z)) > tol and iter < 100:
  iter += 1
  what_old = np.copy(what)
  what,Z,_,_ = updatewagehat(what,theta=theta,N=N,X=X_sq,psi=0.1)
  if iter % 10 == 0:
    print(iter)
    print(Z)
    print(what)

print("Done?")

what,Z,Phat,X1 = updatewagehat(what,theta=4,N=3,X=X_sq,psi=0.1)
print(Z)
print(what)


10
[ 8.34014292e-05 -4.22723491e-05 -4.22723491e-05]
[1.00868191 0.99694031 0.99694031]
Done?
[ 4.58753609e-05 -2.32904601e-05 -2.32904601e-05]
[1.00869963 0.99693413 0.99693413]


さてこれが均衡を計算しなおしたと一致するか確認してみる
$$ \hat{U}_i = \hat{w}_i / \hat{P}_i $$

In [ ]:
Uhat = what / Phat
Uhat_rs = U_cf/U_sq
what_rs = w_cf/w_sq

print(Uhat)
print(Uhat_rs)
print(what)
print(what_rs)

[1.10945031 1.08091286 1.08091286]
[1.10938209 1.08093633 1.08093633]
[1.00869963 0.99693413 0.99693413]
[1.00861942 0.99696227 0.99696227]
